### OCI Data Science - Useful Tips
Everything stored in the <span style="background-color: #d5d8dc ">/home/datascience</span> folder is now stored on your block volume drive. The <span style="background-color: #d5d8dc ">ads-examples</span> folder has moved outside of your working space. Notebook examples are now accessible through a Launcher tab "Notebook Examples" button.
<details>
<summary><font size="2">1. Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">2. OCI Configuration and Key Files Set Up</font></summary><p>Follow the instructions in the getting-started notebook. That notebook is accessible via the "Getting Started" Launcher tab button.</p>
</details>
<details>
<summary><font size="2">3. Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">4. Typical Cell Imports and Settings</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import MLData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">5. Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import logging
import os
import cx_Oracle

In [2]:
# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = "/home/datascience/ADB"
connection = cx_Oracle.connect('hackathon', 'bCfrCC0981267', 'adw_low')

In [3]:

# Validação duplicidade probe

Query_PROBE = """ select count(*) from (SELECT CD_POSICAO, NUMBERX_CRIPT, COUNT(*) FROM 
            (SELECT DISTINCT A.* FROM EVE_PROBE_MANU_HCKT A  
                INNER JOIN 
                (SELECT CD_POSICAO, NUMBERX_CRIPT, MAX(RSAC_DATE_REF) AS DT_MAX 
                    FROM EVE_PROBE_MANU_HCKT 
                WHERE rsac_event_cycle_point =  1
                GROUP BY CD_POSICAO, NUMBERX_CRIPT
                ) B
            ON A.NUMBERX_CRIPT = B.NUMBERX_CRIPT
            AND A.RSAC_DATE_REF = B.DT_MAX
            ) where cd_posicao = 202001
             group by CD_POSICAO, NUMBERX_CRIPT
             having count(*) > 1
             )
            """


PROBE = pd.read_sql(Query_PROBE, con=connection) 
PROBE.head()

,COUNT(*)
0,0


In [5]:

# Query para levantarmos, consumo e uso da buffer sem uso cartão.

Query_CUBO_BUFFER_TCKTMEDIO = """ 
            
            SELECT  A.CD_POSICAO, 
                    rsac_curr_status AS STATUS,
                    CASE WHEN D.CPF_CRIP IS NOT NULL THEN 1 ELSE 0 END AS USO_BUFFER, 
                    CASE WHEN GACD_FG_ACESS_APP_CRF > 0 THEN 1 ELSE 0 END AS FL_USO_APP, 
                    CASE WHEN rsac_nbr_purchases > 0 THEN 1 ELSE 0 END AS FL_COMPRA_CICLO, 
                    SUM(rsac_value_purchases_l6m) AS VL_COMPRA_ULT6M,
                    SUM(FATURAMENTO) AS FATURAMENTO_BUFFER,
                    SUM(rsac_current_credit_limit) AS LIMITE,         
                    COUNT(*) AS QTDE
            FROM 
            (SELECT DISTINCT A.* FROM EVE_PROBE_MANU_HCKT A  
                INNER JOIN 
                (SELECT CD_POSICAO, NUMBERX_CRIPT, MAX(RSAC_DATE_REF) AS DT_MAX 
                    FROM EVE_PROBE_MANU_HCKT 
                WHERE rsac_event_cycle_point =  1
                GROUP BY CD_POSICAO, NUMBERX_CRIPT
                ) B
            ON A.NUMBERX_CRIPT = B.NUMBERX_CRIPT
            AND A.RSAC_DATE_REF = B.DT_MAX
            )  A INNER JOIN 
            
            ( select a.* from EVE_CHAVES_MANU_HCKT a inner join 
                                            (SELECT CACCSERNO_CRIPT, COUNT(*) AS QTDE FROM EVE_CHAVES_MANU_HCKT 
                                            GROUP BY CACCSERNO_CRIPT HAVING COUNT(*) = 1 
                                            ) b on a.CACCSERNO_CRIPT = b.CACCSERNO_CRIPT  and QTDE = 1
                                        ) B
            ON A.NUMBERX_CRIPT = B.NUMBERX_CRIPT
            
            LEFT JOIN 
            (SELECT OSU_DT_POSICAO, CACCSERNO_CRIPT, MAX(GACD_FG_ACESS_APP_CRF) AS GACD_FG_ACESS_APP_CRF FROM EVE_CROSSCANAIS_MANU_HCKT GROUP BY OSU_DT_POSICAO, CACCSERNO_CRIPT) C
                         ON B.CACCSERNO_CRIPT = C.CACCSERNO_CRIPT
                         AND A.CD_POSICAO = C.OSU_DT_POSICAO
                         
            LEFT JOIN              
            (
             SELECT ANOMES, CPF_CRIP, SUM(TICU_VL_TOTALITEM) AS FATURAMENTO
             FROM EVE_BUFFER_CCI_MANU_HCKT
             GROUP BY ANOMES, CPF_CRIP
             ) D
             ON A.CD_POSICAO = D.ANOMES
             AND B.CPF_CRIP = D.CPF_CRIP
            WHERE B.CD_EMPRESA = 1
            AND rsac_curr_status IN( 'NORM', 'DLNQ')
            GROUP BY A.CD_POSICAO, 
                     CASE WHEN D.CPF_CRIP IS NOT NULL THEN 1 ELSE 0 END , 
                     CASE WHEN GACD_FG_ACESS_APP_CRF > 0 THEN 1 ELSE 0 END,
                     rsac_curr_status,
                     CASE WHEN rsac_nbr_purchases > 0 THEN 1 ELSE 0 END
                     
            
            
            
             
            """



CUBO_USO_BFF_TCKT = pd.read_sql(Query_CUBO_BUFFER_TCKTMEDIO, con=connection) 
CUBO_USO_BFF_TCKT.head()
CUBO_USO_BFF_TCKT.to_csv(r'CUBO_HACKATON.csv', index = False, header=True)

,CD_POSICAO,STATUS,USO_BUFFER,FL_USO_APP,FL_COMPRA_CICLO,VL_COMPRA_ULT6M,FATURAMENTO_BUFFER,LIMITE,QTDE
0,202003,NORM,1,1,1,8690515.19,912096.44,24044907,1596
1,202003,DLNQ,1,0,1,347101.07,34640.64,663538,100
2,202006,NORM,1,1,1,13128418.55,1542082.03,39175139,2571
3,202007,DLNQ,1,0,0,101230.83,38445.47,643765,86
4,202009,NORM,1,1,1,19087595.34,2337190.70,59120626,3817


In [6]:

# Query para levantar o uso do APP, com a base de fechamento (Sumário Conta)

Query_USO_APP = """ 
SELECT  DATE_REF, SUM(CASE WHEN B.GACD_FG_ACESS_APP_CRF > 0 THEN 1 ELSE 0 END) AS USOAPP, COUNT(*) AS QTD FROM
                (SELECT A.* FROM EVE_SUMARIOCONTA_MANU_HCKT A  INNER JOIN   
                         (SELECT   
                         DATE_REF, MAX(PRAM_NU_ANOMESSAFRA) AS PRAM_NU_ANOMESSAFRA, CACCSERNO_CRIPT   
                         FROM EVE_SUMARIOCONTA_MANU_HCKT  
                         GROUP BY DATE_REF, CACCSERNO_CRIPT
                         ) B  
                         ON A.CACCSERNO_CRIPT = B.CACCSERNO_CRIPT   
                         AND A.PRAM_NU_ANOMESSAFRA = B.PRAM_NU_ANOMESSAFRA 
                         AND A.DATE_REF = B.DATE_REF) A
                         LEFT JOIN (SELECT OSU_DT_POSICAO, CACCSERNO_CRIPT, MAX(GACD_FG_ACESS_APP_CRF) AS GACD_FG_ACESS_APP_CRF FROM EVE_CROSSCANAIS_MANU_HCKT GROUP BY OSU_DT_POSICAO, CACCSERNO_CRIPT) B
                         ON A.CACCSERNO_CRIPT = B.CACCSERNO_CRIPT
                         AND A.DATE_REF = B.OSU_DT_POSICAO
                         LEFT JOIN ( select a.* from EVE_CHAVES_MANU_HCKT a inner join 
                                            (SELECT CACCSERNO_CRIPT, COUNT(*) AS QTDE FROM EVE_CHAVES_MANU_HCKT 
                                            GROUP BY CACCSERNO_CRIPT HAVING COUNT(*) = 1 
                                            ) b on a.CACCSERNO_CRIPT = b.CACCSERNO_CRIPT  and QTDE = 1
                                        ) C 
                         ON A.CACCSERNO_CRIPT = C.CACCSERNO_CRIPT 
                         AND A.DATE_REF = B.OSU_DT_POSICAO
                         WHERE CD_EMPRESA = 1 
                GROUP BY DATE_REF
                        
        """

CUBO_USO_APP = pd.read_sql(Query_USO_APP, con=connection) 
CUBO_USO_APP.head()

CUBO_USO_APP.to_csv(r'CUBO_HACKATON2.csv', index = False, header=True, sep = '|')

,DATE_REF,USOAPP,QTD
0,202006,7878,36579
1,202007,8158,36597
2,202002,6825,36513
3,202008,8553,36656
4,202004,7774,36538


In [7]:

# Na query abaixo utilizamos o bacen para levantar a preferência pelo cartão carrefour, baseado no saldo faturado.

Query_BACEN = """ 

SELECT 
 CASE WHEN  NVL(SALDO_MERC,0) - NVL(SALDO_BANCO,0) < 0 THEN 1
      WHEN  NVL(SALDO_BANCO,0)/NVL(SALDO_MERC,0) > 0.5 THEN 1 ELSE 0 END AS PREFERENCIA_BANCO, COUNT(*) AS QT
FROM 
(
SELECT 
NUMBERX_CRIPT,
SUM(
rsac_current_balance

) AS SALDO_BANCO
FROM 
    (SELECT DISTINCT A.* FROM EVE_PROBE_MANU_HCKT A  
                    INNER JOIN 
                    (SELECT CD_POSICAO, NUMBERX_CRIPT, MAX(RSAC_DATE_REF) AS DT_MAX 
                        FROM EVE_PROBE_MANU_HCKT 
                    WHERE rsac_event_cycle_point =  1
                    AND CD_POSICAO = 202008
                    AND rsac_current_balance > 0 
                    GROUP BY CD_POSICAO, NUMBERX_CRIPT
                    ) B
                ON A.NUMBERX_CRIPT = B.NUMBERX_CRIPT
                AND A.RSAC_DATE_REF = B.DT_MAX
                ) 
    GROUP BY NUMBERX_CRIPT) A 

            INNER JOIN 
            
            ( select a.* from EVE_CHAVES_MANU_HCKT a inner join 
                                            (SELECT  CACCSERNO_CRIPT, COUNT(*) AS QTDE FROM EVE_CHAVES_MANU_HCKT 
                                            GROUP BY CACCSERNO_CRIPT HAVING COUNT(*) = 1 
                                            ) b on a.CACCSERNO_CRIPT = b.CACCSERNO_CRIPT  and QTDE = 1
                                        ) B
            ON A.NUMBERX_CRIPT = B.NUMBERX_CRIPT
            
            INNER JOIN 
            
            
            (
            SELECT  cpf_crip,
                    SUM(card_v110) /4  AS SALDO_MERC FROM EVE_BACEN_MANU_HCKT
            
            GROUP BY cpf_crip

            
            ) C 
            ON B.CPF_CRIP = C.cpf_crip
            
            GROUP BY CASE WHEN  NVL(SALDO_MERC,0) - NVL(SALDO_BANCO,0) < 0 THEN 1
                          WHEN  NVL(SALDO_BANCO,0)/NVL(SALDO_MERC,0) > 0.5 THEN 1 ELSE 0 END
            
            
            """


CUBO_BACEN = pd.read_sql(Query_BACEN, con=connection) 
CUBO_BACEN.head()

,PREFERENCIA_BANCO,QT
0,1,54074
1,0,82816
